# Advances Prompt Engineering Strategies

### Loading OpenAI API Key

In [2]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv(dotenv_path='../.env')  # Specify the path to your .env file

# Access the environment variable
api_key = os.getenv('OPENAI_API_KEY')

# Check if the variable is loaded
if api_key or api_key == "":
    print("API key loaded successfully.")
else:
    print("Failed to load API key.")

API key loaded successfully.


## Initializing OpenAI API

In [3]:
from openai import OpenAI

client = OpenAI(api_key=api_key)

## Auxiliary Functions

### Function to get the model response

In [20]:
class Message:
    def __init__(self, role, content):
        self.role = role
        self.content = content

def get_response(messages:list[Message], model="gpt-3.5-turbo", temperature=0.7):
    """
    Get a response from the OpenAI API.

    Parameters:
    - prompt (str): The input prompt for the model.
    - model (str): The model to use. Default is "gpt-3.5-turbo".
    - temperature (float): Sampling temperature. Default is 0.7.

    Returns:
    - str: The model's response.
    """
    model_response = client.chat.completions.create(
        model=model,
        messages=[message.__dict__ for message in messages],
        temperature=temperature
    )

    # Access the content of the first choice
    return model_response.choices[0].message.content

# Shots

## Zero-Shot, One-Shot, and Few-Shot Prompts

### Function to generate a prompt for extracting odd numbers

In [21]:
def get_extract_odd_numbers_prompt(input_data:set[int], examples:int=0):
    """
    Generate a prompt for the OpenAI API to extract odd numbers from a set of integers.
    :param input_data: A set of integers.
    :param examples: Number of examples to include in the prompt.
    :return: A string containing the prompt.
    """
    def get_question(question_data:set[int]):
        return f"Q: What are the odd numbers in the given set of integers {question_data}?"
    extract_odds_prompt = f"{get_question(input_data)}. A: "
    if examples > 0:
        shots = ""
        for i_exp in range(examples):
            example_data = set(range(i_exp * 2 + 1, i_exp * 2 + 3))
            odds = set(filter(lambda x: x % 2 != 0, example_data))
            shots += f"{get_question(example_data)}. A: {odds}\n"
        return shots + extract_odds_prompt
    return f"""{extract_odds_prompt}"""



In [22]:
# Example usage (only prompt)
data = {1, 2, 3, 4, 5, 6, 7, 8, 9}

for i in range(1, 4):
    prompt = get_extract_odd_numbers_prompt(data, examples=i)
    print(f"- Prompt with {i} example(s):\n{prompt}")
    print("-" * 50)

- Prompt with 1 example(s):
Q: What are the odd numbers in the given set of integers {1, 2}?. A: {1}
Q: What are the odd numbers in the given set of integers {1, 2, 3, 4, 5, 6, 7, 8, 9}?. A: 
--------------------------------------------------
- Prompt with 2 example(s):
Q: What are the odd numbers in the given set of integers {1, 2}?. A: {1}
Q: What are the odd numbers in the given set of integers {3, 4}?. A: {3}
Q: What are the odd numbers in the given set of integers {1, 2, 3, 4, 5, 6, 7, 8, 9}?. A: 
--------------------------------------------------
- Prompt with 3 example(s):
Q: What are the odd numbers in the given set of integers {1, 2}?. A: {1}
Q: What are the odd numbers in the given set of integers {3, 4}?. A: {3}
Q: What are the odd numbers in the given set of integers {5, 6}?. A: {5}
Q: What are the odd numbers in the given set of integers {1, 2, 3, 4, 5, 6, 7, 8, 9}?. A: 
--------------------------------------------------


In [23]:
# Example usage (with response)
data = {3, 5, 11, 12, 16}
zero_shot_prompt = get_extract_odd_numbers_prompt(data)
response = get_response(messages=[Message(role="user", content=zero_shot_prompt)])
print(f"\nPrompt zero shot:\n{zero_shot_prompt}")
print(f"\nResponse zero shot:\n{response}")
print("-" * 50)

one_shot_prompt = get_extract_odd_numbers_prompt(data, examples=1)
response = get_response(messages=[Message(role="user", content=one_shot_prompt)])
print(f"\nPrompt one shot:\n{one_shot_prompt}")
print(f"\nResponse one shot:\n{response}")
print("-" * 50)

few_shot_prompt = get_extract_odd_numbers_prompt(data, examples=2)
response = get_response(messages=[Message(role="user", content=few_shot_prompt)])
print(f"\nPrompt few shot:\n{few_shot_prompt}")
print(f"\nResponse few shot:\n{response}")
print("-" * 50)


Prompt zero shot:
Q: What are the odd numbers in the given set of integers {16, 3, 5, 11, 12}?. A: 

Response zero shot:
3, 5, 11
--------------------------------------------------

Prompt one shot:
Q: What are the odd numbers in the given set of integers {1, 2}?. A: {1}
Q: What are the odd numbers in the given set of integers {16, 3, 5, 11, 12}?. A: 

Response one shot:
{3, 5, 11}
--------------------------------------------------

Prompt few shot:
Q: What are the odd numbers in the given set of integers {1, 2}?. A: {1}
Q: What are the odd numbers in the given set of integers {3, 4}?. A: {3}
Q: What are the odd numbers in the given set of integers {16, 3, 5, 11, 12}?. A: 

Response few shot:
{3, 5, 11}
--------------------------------------------------


### Sentiment analysis with few-shot prompting

#### Function to generate messages for sentiment analysis

In [34]:
def get_sentiment_analysis_messages(input_text:str, examples:int=0):
    """
    Generate messages for sentiment analysis.
    :param input_text: The input text for sentiment analysis.
    :param examples: Number of examples to include in the prompt.
    :return: A list of messages for the OpenAI API.
    """
    if examples > 6:
        raise ValueError("The maximum number of examples is 6.")

    messages = []
    if examples > 0:
        sentiment_examples = [
            {"sentence": "I love programming!", "sentiment": "Positive"},
            {"sentence": "I hate programming!", "sentiment": "Negative"},
            {"sentence": "This is a great day", "sentiment": "Positive"},
            {"sentence": "This is a bad day", "sentiment": "Negative"},
            {"sentence": "I am very happy.", "sentiment": "Positive"},
            {"sentence": "I am very sad.", "sentiment": "Negative"}
        ]
        for i_exp in range(examples):
            # Example data, question as user
            msg = Message(role="user", content=sentiment_examples[i_exp].get("sentence"))
            messages.append(msg)
            # Example data, answer as assistant
            msg = Message(role="assistant", content=sentiment_examples[i_exp].get("sentiment"))
            messages.append(msg)

    question_msg = Message(role="user", content=input_text)
    messages.append(question_msg)
    return messages

In [35]:
# Example usage (only prompt)
text = "I love programming!"
for i in range(1, 5):
    msgs = get_sentiment_analysis_messages(text, examples=i)
    print(f"\nMsgs with {i} example(s):\n{[msg.__dict__ for msg in msgs]}")
    print("-" * 50)


Msgs with 1 example(s):
[{'role': 'user', 'content': 'I love programming!'}, {'role': 'assistant', 'content': 'Positive'}, {'role': 'user', 'content': 'I love programming!'}]
--------------------------------------------------

Msgs with 2 example(s):
[{'role': 'user', 'content': 'I love programming!'}, {'role': 'assistant', 'content': 'Positive'}, {'role': 'user', 'content': 'I hate programming!'}, {'role': 'assistant', 'content': 'Negative'}, {'role': 'user', 'content': 'I love programming!'}]
--------------------------------------------------

Msgs with 3 example(s):
[{'role': 'user', 'content': 'I love programming!'}, {'role': 'assistant', 'content': 'Positive'}, {'role': 'user', 'content': 'I hate programming!'}, {'role': 'assistant', 'content': 'Negative'}, {'role': 'user', 'content': 'This is a great day'}, {'role': 'assistant', 'content': 'Positive'}, {'role': 'user', 'content': 'I love programming!'}]
--------------------------------------------------

Msgs with 4 example(s):


In [37]:
# Example usage (with response)
temp = 0.7
text = "The price of the product is really fair given its features"
zero_shot_msgs = get_sentiment_analysis_messages(text)
response = get_response(messages=zero_shot_msgs, temperature=temp)
print(f"\nResponse zero shot:\n{response}")
for zero_shot_msgs in zero_shot_msgs:
    print(f"- {zero_shot_msgs.__dict__}")
print("-" * 50)

one_shot_msgs = get_sentiment_analysis_messages(text, examples=1)
response = get_response(messages=one_shot_msgs, temperature=temp)
print(f"\nResponse one shot:\n{response}")
for one_shot_msgs in one_shot_msgs:
    print(f"- {one_shot_msgs.__dict__}")
print("-" * 50)

few_shot_msgs = get_sentiment_analysis_messages(text, examples=2)
response = get_response(messages=few_shot_msgs, temperature=temp)
print(f"\nResponse few shot:\n{response}")
for few_shot_msgs in few_shot_msgs:
    print(f"- {few_shot_msgs.__dict__}")
print("-" * 50)


Response zero shot:
 and quality. I believe it offers great value for the money.
- {'role': 'user', 'content': 'The price of the product is really fair given its features'}
--------------------------------------------------

Response one shot:
Positive
- {'role': 'user', 'content': 'I love programming!'}
- {'role': 'assistant', 'content': 'Positive'}
- {'role': 'user', 'content': 'The price of the product is really fair given its features'}
--------------------------------------------------

Response few shot:
Positive
- {'role': 'user', 'content': 'I love programming!'}
- {'role': 'assistant', 'content': 'Positive'}
- {'role': 'user', 'content': 'I hate programming!'}
- {'role': 'assistant', 'content': 'Negative'}
- {'role': 'user', 'content': 'The price of the product is really fair given its features'}
--------------------------------------------------


# Steps

## Single Step vs Multi-Step Prompts

### Single Step Prompt

In [39]:
single_prompt = """Make a plan for a beach vacation."""
response = get_response(messages=[Message(role="user", content=single_prompt)])
print(f"\nSingle-Step Prompt:\n{single_prompt}")
print(f"\nResponse:\n{response}")


Single-Step Prompt:
Make a plan for a beach vacation.

Response:
1. Choose a destination: Research different beach destinations and choose one that fits your preferences, budget, and travel time.

2. Book accommodations: Find a beachfront hotel, resort, or vacation rental that suits your needs and book your accommodations in advance to secure the best rates.

3. Plan transportation: Arrange for flights, car rentals, or other transportation to get to your beach destination. Consider renting a car if you plan to explore the area during your vacation.

4. Pack essentials: Make a packing list of essentials such as sunscreen, beach towels, swimsuits, hats, sunglasses, and beach toys. Don't forget to pack any medications, toiletries, and other personal items you may need.

5. Plan activities: Research and plan activities to do during your beach vacation, such as water sports, boat tours, beach picnics, hiking, or sightseeing. Make reservations for any activities or tours in advance to ensur

### Multi-Step Prompt

In [40]:
single_prompt = """Make a plan for a beach vacation as follows:
Step 1: List four potential locations.
Step 2: For each location, provide accommodation options.
Step 3: For each location, suggest activities.
Step 4: For each location, evaluate the pros and cons.
"""
response = get_response(messages=[Message(role="user", content=single_prompt)])
print(f"\nMulti-Step Prompt:\n{single_prompt}")
print(f"\nResponse:\n{response}")


Multi-Step Prompt:
Make a plan for a beach vacation as follows:
Step 1: List four potential locations.
Step 2: For each location, provide accommodation options.
Step 3: For each location, suggest activities.
Step 4: For each location, evaluate the pros and cons.


Response:
Step 1: 
1. Maui, Hawaii
2. Cancun, Mexico
3. Myrtle Beach, South Carolina
4. Bali, Indonesia

Step 2:

Maui, Hawaii:
- Resort hotels
- Vacation rentals
- Beachfront condos

Cancun, Mexico:
- All-inclusive resorts
- Boutique hotels
- Airbnb rentals

Myrtle Beach, South Carolina:
- Oceanfront hotels
- Beach houses
- Campgrounds

Bali, Indonesia:
- Luxury resorts
- Villas
- Eco-friendly lodges

Step 3:

Maui, Hawaii:
- Snorkeling
- Surfing
- Hiking in Haleakalā National Park
- Luau

Cancun, Mexico:
- Visiting Mayan ruins
- Water sports
- Swimming with dolphins
- Nightlife in the Hotel Zone

Myrtle Beach, South Carolina:
- Golfing
- Fishing
- Shopping at Broadway at the Beach
- Water parks

Bali, Indonesia:
- Yoga ret

In [43]:
code = '''
def calculate_rectangle_area(length, width):
    area = length * width
    return area
'''

prompt = f"""
Analyze the function delimited by triple backticks and determine if it is correct as follows:
Step 1: Correct syntax
Step 2: Receiving two inputs
Step 3: Returning one output
```{code}```
"""

response = get_response(messages=[Message(role="user", content=prompt)])
print(f"\nMulti-Step Prompt:\n{prompt}")
print(f"\nResponse:\n{response}")


Multi-Step Prompt:

Analyze the function delimited by triple backticks and determine if it is correct as follows:
Step 1: Correct syntax
Step 2: Receiving two inputs
Step 3: Returning one output
```
def calculate_rectangle_area(length, width):
    area = length * width
    return area
```


Response:
Step 1: The syntax of the function is correct. 
Step 2: The function is receiving two inputs: 'length' and 'width'. 
Step 3: The function is correctly returning one output which is the calculated area of the rectangle formed by multiplying the length and width. 

Therefore, the function is correct in terms of syntax, inputs, and output.
